In [7]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader
import torch.nn.functional as F
from torch_geometric.utils.convert import from_scipy_sparse_matrix, from_networkx
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm
import torch.nn as nn


import random
import numpy as np
from scipy.io import loadmat, savemat
import pandas as pd
import scipy.special as SS
import scipy.stats as SSA
import copy
import math
from sklearn.model_selection import ParameterGrid
import os
import numpy.linalg as LA
import gzip
from scipy import sparse
from torch.utils.data import random_split
import time

# load pickle module
import pickle
import networkx as nx
# from tqdm import tqdm
import sys
import h5py

In [8]:
%run ../../codes/GCN.py

In [9]:
lr_list = [np.power(0.5, i) for i in range(2, 16, 2)]*10

In [10]:
lr_list[3]

0.00390625

In [12]:
class GCN_ng(torch.nn.Module):  # the simpliest model that GNN and it is classical, used as baseline
    def __init__(self, num_node_features):
        super(GCN_ng, self).__init__()
        self.conv1 = GCNConv(num_node_features, 128)
        self.conv2 = GCNConv(128, 64)
        self.conv3 = GCNConv(64, 16)
        self.conv4 = GCNConv(16, 8)
        self.fc = torch.nn.Linear(8, 1)

    """
        hyperparameters:
        - number of hidden layers
        - number of hidden channels
        - dropout rate (now it's zero)
        - learning rate <- most important to tune
        - weight decay
        - etc etc.
    """

    def forward(self, data):
        # x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr, data.batch
        x, batch = data.x, data.batch
        edge_index = torch.Tensor([[], []]).to(x.device).long()

        x = self.conv1(x, edge_index)
        x = F.elu(x)

        x = self.conv2(x, edge_index)
        x = F.elu(x)

        x = self.conv3(x, edge_index)
        x = F.elu(x)

        x = self.conv4(x, edge_index)
        x = F.elu(x)

        x = global_mean_pool(x, batch)
        x = self.fc(x)

        return x

In [14]:
datafolder = '/Users/qingyao/../../Volumes/Seagate_Qing/branching_data/gnn_regression_1d5/including_large_r/'
    # datafolder = '/Users/qingyao/Documents/branching_data/gnn_regression/'

num_x = 50
layer = 4
# save_dir = '/rds/general/user/qy1815/home/branching_superspreading/regression_ng_{}_layer_{}/'.format(
#num_x, layer)

job_idx = 100 #int(sys.argv[1])-1  # ensemble/parameter index
s = job_idx//70
es_idx = 4 # job_idx % 70  # ensemble/parameter index
s = 1 #int(s)  # seed index

    # load data
    # WN = np.loadtxt('W_avg.csv')
dataset = torch.load(datafolder+'dataset_{}.pt'.format(num_x))
lr_list = [np.power(0.5, i) for i in range(2, 16, 2)]*10
my_lr = 0.0001 #lr_list[es_idx]

torch.manual_seed(s)
all_data_len = len(dataset)
train_size = int(all_data_len * 0.6)
val_size = int(all_data_len * 0.2)
test_size = all_data_len - train_size - val_size

train_data, val_data, test_data = random_split(
        dataset, [train_size, val_size, test_size])

    # Now we can create a DataLoader
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=True)
    # Create a model and an optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model = GCN_ng(num_node_features=num_x, hidden_channels=[
#         128, 64, 16, 8], num_hlayers=layer).to(device)

model = GCN_ng(num_node_features=num_x).to(device)
# model = MLP(num_node_features=num_x, hidden_channels=[
#         128, 64, 16, 8], num_hlayers=layer).to(device)
optimizer = torch.optim.Adam(
        model.parameters(), lr=my_lr, weight_decay=5e-4)

    # training and validation
counter = 0
count_epochs = 0
best = float("inf")
epochs = 200
patience = 15
loss_ep = []

seed = int(time.time()) + es_idx
# random.seed(seed)
torch.manual_seed(seed)

for epoch in range(epochs):
    loss, myres, reals = train(model, train_loader, optimizer, device)
    # loss_ep.append(loss)
    val_loss = validate(model, val_loader, device)
    loss_ep.append(val_loss)
    print(epoch, val_loss)
    if val_loss < best:
        best = val_loss
        counter = 0
        # Save the best model
#         torch.save(model.state_dict(
#         ), save_dir+'best_model_{}_{}.pth'.format(s, es_idx))
    else:
        counter += 1
        count_epochs += 1

    if counter > patience:
        break

0 4.080893083602663
1 4.0583167182074655
2 4.042672326829698
3 4.030082913050576
4 4.019350860989283
5 4.007984139427307
6 3.994661515099662
7 3.978609083569239
8 3.9599446682702926
9 3.940372591170054
10 3.912268889139569
11 3.8796517251029847
12 3.8369974295298257
13 3.7876985277448383
14 3.7263285402267696
15 3.6581817929706877
16 3.569167208293128
17 3.476284569028824
18 3.3527408130585203
19 3.2428363709222703
20 3.0950317148178343
21 2.9746818481929718
22 2.8076240085420157
23 2.6712691435738214
24 2.5228989646548317
25 2.3777884869348433
26 2.2436788944970996
27 2.1253075648867896
28 2.013202729679289
29 1.9428679893887233
30 1.8198808231051007
31 1.765735440027146
32 1.6931871599621242
33 1.6526289103523133
34 1.63714059534527
35 1.588583771387736
36 1.5657230059305827
37 1.5516878502709526
38 1.5160208702087403
39 1.5015594149392748
40 1.4921401693707421
41 1.4853430819889857
42 1.467276732505314
43 1.5052682373258803
44 1.4637004080272855
45 1.4358827613648915
46 1.4271565032